In [1]:
#installing kaggle library
!pip install kaggle

In [2]:
#configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

importing the twitter sentiment dataset

In [3]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 93% 75.0M/80.9M [00:00<00:00, 779MB/s]
100% 80.9M/80.9M [00:00<00:00, 769MB/s]


In [4]:
#extracting the compressed dataset
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

The dataset is extracted


Importing the dependencies

In [5]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [6]:
#importing stop words- words that doesnt have much importance, we are using this since since we have alarge data we reduce the data using stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
#printing stopwords in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data processing

In [8]:
 #loading the data
 twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [9]:
twitter_data.shape

(1599999, 6)

In [10]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [11]:
#changing the columns and reading the again
column_names=['target','ids','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')

In [12]:
twitter_data.shape

(1600000, 6)

In [13]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [14]:
#counting the number of missing values in the dataset
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [15]:
#checking the distrubution of target column
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Changing target label 4 to 1

In [16]:
twitter_data.replace({'target':{4:1}},inplace=True) # inplae so that it makes changes in the manin data

In [17]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


0-----NEgative tweet
1-----Postive Tweet

Using Stemming
stemming is the process of reducing a word to its root word

In [18]:
port_stem=PorterStemmer()

In [23]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-z]',' ',content) #remove the puntuation special characters
  stemmed_content=stemmed_content.lower() #convert to lower letter
  stemmed_content=stemmed_content.split() #split all the words into the list
  stemmed_content=[port_stem.stem(word)for word in stemmed_content if not word in stopwords.words('english')] #perform operation of it is not a stop word
  stemmed_content=' '.join(stemmed_content) #then join the words

  return stemmed_content

In [24]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [25]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [26]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [27]:
#sperating data and label
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [28]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [29]:
print(Y)

[0 0 0 ... 1 1 1]


In [30]:
#splitting the data into training and testing data
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2) #stratify to split the negative and postive tweet in equal proportion

In [31]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


In [32]:
#converting the textual data into numerical data

vectorizer = TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)

In [33]:
print(X_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 9416150 stored elements and shape (1280000, 483851)>
  Coords	Values
  (0, 458421)	0.27228112326228093
  (0, 372989)	0.3583819096048567
  (0, 194823)	0.5274064910069903
  (0, 116318)	0.3749338694119814
  (0, 247667)	0.4215060595161854
  (0, 464940)	0.4483581441053542
  (1, 169453)	1.0
  (2, 116318)	0.45905705320412793
  (2, 132152)	0.1891875792321668
  (2, 427967)	0.18706371586911208
  (2, 137276)	0.29069581876534506
  (2, 427034)	0.32103058232997417
  (2, 455221)	0.3296073093697297
  (2, 83506)	0.31303964332256906
  (2, 465311)	0.33480686186877023
  (2, 281186)	0.24137202118772233
  (2, 429906)	0.15168139293444055
  (2, 187474)	0.16194060015844067
  (2, 159240)	0.18800868496584808
  (2, 140239)	0.20289994039686138
  (2, 304223)	0.1678648733603879
  (3, 427034)	0.29025988953598614
  (3, 167435)	0.44570129071167713
  (3, 160331)	0.2785087322306358
  (3, 61032)	0.52009780709937
  :	:
  (1279996, 335154)	0.21190629789318696
  (

In [34]:
print(X_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2275549 stored elements and shape (320000, 483851)>
  Coords	Values
  (0, 17977)	0.17180827032540266
  (0, 34872)	0.16236726920078096
  (0, 72943)	0.2678484639307855
  (0, 112919)	0.367081693610678
  (0, 140294)	0.25509664355434203
  (0, 146252)	0.23682534421364126
  (0, 180544)	0.2803743470609849
  (0, 285841)	0.4532311769636366
  (0, 294440)	0.17813419553395948
  (0, 408024)	0.21977439030363208
  (0, 419078)	0.34884649404705365
  (0, 429906)	0.3140986667345934
  (0, 442233)	0.17903569516819148
  (1, 8801)	0.30376315944222493
  (1, 17977)	0.2084958588978494
  (1, 153709)	0.5722666578553444
  (1, 229396)	0.40557814327518943
  (1, 270721)	0.28432607616832933
  (1, 366102)	0.4820488545932507
  (1, 385085)	0.24302725359248564
  (2, 25826)	0.3529227215218366
  (2, 38175)	0.37941511785250515
  (2, 95488)	0.363168936008175
  (2, 192882)	0.5885804190746107
  (2, 270795)	0.2562344190631401
  :	:
  (319994, 465688)	0.2780204101679032

In [38]:
#training the model

#LogisticRegreesion

model=LogisticRegression(max_iter=1000)


In [39]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

In [40]:
#accuracy_score on the training data
X_train_prediction=model.predict(X_train)
training_accuracy_score=accuracy_score(X_train_prediction,Y_train)
print('Accuracy score on training data :',training_accuracy_score)

Accuracy score on training data : 0.79812734375


In [42]:
#accuracy_score on the test data
X_test_prediction=model.predict(X_test)
test_accuracy_score=accuracy_score(X_test_prediction,Y_test)
print('Accuracy score on test data :',test_accuracy_score)

Accuracy score on test data : 0.777265625


In [43]:
#Saving the model
import pickle

filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))

using the model for future prediction

In [44]:
#loding the model

load_model=pickle.load(open('/content/trained_model.sav','rb'))

In [45]:
X_new=X_test[200]
print(Y_test[200])

prediction=load_model.predict(X_new)
print(prediction)

if(prediction[0]==0):
  print("negative tweet")
else:
  print("postive Tweet")

1
[1]
postive Tweet
